In [40]:
import pandas as pd
from pulp import *

from collections import defaultdict

In [41]:
df_distance_km = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')
df_location = pd.read_excel('../../Datos_P1/df_location.xlsx')
df_orders = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_vehicle = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')


In [42]:
df_vehiculos_aleatorios = df_vehicle.sample(n=3, random_state=420) #Elección aleatoria de 3 vehiculos del dataframe, con un random_state para que sea más consistente
df_vehiculos_aleatorios.reset_index(drop=True ,inplace=True) #Resetear el index del nuevo dataframe para que tenga los numeros 0, 1 y 2
df_vehiculos_aleatorios

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,5,10000,0.32,350
1,6,3129,0.14,791
2,4,3321,0.19,514


Esto te elige 3 vehiculos aleatorios del dataframe de vehiculos, lo he dejado con un random_state para que sea consistente, aunque deberiamos probarlo sin ello más adelante

In [43]:
# Preparación datos para PuLP
capacidades = dict(zip(df_vehiculos_aleatorios["vehiculo_id"], df_vehiculos_aleatorios["capacidad_kg"]))
costos = dict(zip(df_vehiculos_aleatorios["vehiculo_id"], df_vehiculos_aleatorios["costo_km"]))
autonomias = dict(zip(df_vehiculos_aleatorios["vehiculo_id"], df_vehiculos_aleatorios["autonomia_km"]))

pedidos = dict(zip(df_orders["cliente"], df_orders["order_demand"]))
pedidos["Almacén"] = 0  # Aseguramos que el almacén no tenga demanda

distancias = df_distance_km.stack().to_dict()  # Convierte a un diccionario (i, j): distancia

clientes = df_orders["cliente"].tolist() + ["Almacén"]  # Incluir el almacén como un cliente especial
vehiculos = df_vehiculos_aleatorios["vehiculo_id"].tolist()

In [44]:
# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos_Mitad", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)
u = LpVariable.dicts("u", [(i, k) for i in clientes for k in vehiculos], lowBound=0)  # Variables auxiliares para flujo

# Función objetivo: minimizar costo total
problema += lpSum(
    x[i, j, k] * costos[k] * distancias.get((i, j), 0)
    for i in clientes for j in clientes for k in vehiculos
)

# Restricciones

# Cada cliente debe ser atendido exactamente una vez
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos) == 1

# Respetar la capacidad de los vehículos
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Flujo continuo: entrada = salida
for k in vehiculos:
    for i in clientes:
        if i != 'Almacén':
            problema += lpSum(x[i, j, k] for j in clientes if j != i) == lpSum(x[j, i, k] for j in clientes if j != i)

# Cada vehículo debe salir del almacén exactamente una vez
for k in vehiculos:
    problema += lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén') == 1

# Cada vehículo debe regresar al almacén exactamente una vez
for k in vehiculos:
    problema += lpSum(x[i, 'Almacén', k] for i in clientes if i != 'Almacén') == 1

# Restringir la autonomía del vehículo
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes) <= autonomias[k]

# Prohibir rutas cíclicas directas (Cliente_X -> Cliente_Y -> Cliente_X)
for k in vehiculos:
    for i in clientes:
        for j in clientes:
            if i != j:
                problema += x[i, j, k] + x[j, i, k] <= 1

# Garantizar la secuencia lógica de rutas mediante variables de flujo
for k in vehiculos:
    for i in clientes:
        for j in clientes:
            if i != 'Almacén' and j != 'Almacén' and i != j:
                problema += u[i, k] - u[j, k] + len(clientes) * x[i, j, k] <= len(clientes) - 1

# Asegurar que un cliente solo sea visitado una vez (refuerzo)
for i in clientes:
    if i != 'Almacén':
        for k in vehiculos:
            problema += lpSum(x[i, j, k] for j in clientes if j != i) <= 1

# Resolver el problema
solver = PULP_CBC_CMD(msg=1, timeLimit=300)
problema.solve(solver)

# Verificar el estado de la solución
print(LpStatus[problema.status])

Infeasible


In [45]:
# Imprimir las rutas resultantes de manera más estructurada
for k in vehiculos:
    print(f"\nVehículo {k}:")
    ruta = []
    for i in clientes:
        for j in clientes:
            if x[i, j, k].value() == 1:
                ruta.append((i, j))
    
    # Ordenar la ruta por cliente de inicio para intentar hacer una secuencia lógica
    ruta.sort(key=lambda x: clientes.index(x[0]))
    
    for segmento in ruta:
        if segmento[0] != segmento[1]:  # Evitar imprimir movimientos a sí mismo
            print(f"  Va de {segmento[0]} a {segmento[1]}")


Vehículo 5:

Vehículo 6:

Vehículo 4:


In [46]:
# Crear un diccionario para almacenar rutas por vehículo
rutas = defaultdict(list)

# Recopilar las rutas
for k in vehiculos:
    for i in clientes:
        for j in clientes:
            if x[i, j, k].value() == 1:
                rutas[k].append((i, j))

# Crear el resumen por vehículo
resumen = []

for k in vehiculos:
    ruta_ordenada = ["Almacén"]  # Inicia desde el almacén
    carga_total = 0
    distancia_total = 0

    # Reconstruir la ruta
    actual = "Almacén"
    visitados = set()

    while rutas[k]:
        encontrado = False
        for origen, destino in rutas[k]:
            if origen == actual and (origen, destino) not in visitados:
                ruta_ordenada.append(destino)
                visitados.add((origen, destino))
                if destino != "Almacén":
                    carga_total += pedidos[destino]  # Sumar la demanda del cliente
                distancia_total += distancias.get((origen, destino), 0)  # Sumar la distancia
                actual = destino
                rutas[k].remove((origen, destino))
                encontrado = True
                break
        if not encontrado:
            break  # Si no encuentra un destino válido, rompe el bucle para evitar ciclos infinitos

    # Datos del vehículo
    capacidad = capacidades[k]
    autonomia = autonomias[k]

    # Guardar en el resumen
    resumen.append({
        "Vehículo": k,
        "Ruta": " -> ".join(ruta_ordenada),
        "Carga total": carga_total,
        "Distancia total (km)": round(distancia_total, 3),
        "Autonomía (km)": autonomia,
        "Capacidad (kg)": capacidad
    })

# Imprimir el resumen
for r in resumen:
    print(f"Vehículo {r['Vehículo']}:")
    print(f"  Ruta: {r['Ruta']}")
    print(f"  Carga total: {r['Carga total']} kg")
    print(f"  Distancia total: {r['Distancia total (km)']} km")
    print(f"  Autonomía: {r['Autonomía (km)']} km")
    print(f"  Capacidad: {r['Capacidad (kg)']} kg")
    print("---")


Vehículo 5:
  Ruta: Almacén
  Carga total: 0 kg
  Distancia total: 0 km
  Autonomía: 350 km
  Capacidad: 10000 kg
---
Vehículo 6:
  Ruta: Almacén
  Carga total: 0 kg
  Distancia total: 0 km
  Autonomía: 791 km
  Capacidad: 3129 kg
---
Vehículo 4:
  Ruta: Almacén
  Carga total: 0 kg
  Distancia total: 0 km
  Autonomía: 514 km
  Capacidad: 3321 kg
---
